## MONGODB-PROJECT: GEOSPATIAL QUERIES

!pip3 install pymongo

In [1]:
from pymongo import MongoClient
import pandas as pd

★ Defino una funcion para conectarme a MongoDB

In [2]:
def connectCollection(database, collection):
    client = MongoClient()
    db = client[database]
    coll = db[collection]
    return db, coll

★ Le digo el database y la coleccion a la que quiero acceder y conectarme

In [3]:
db, coll = connectCollection('companies','companies')

★ Busco todas las compañias que esten en la industria "games_video"

In [4]:
gaming_companies = list(coll.find({'category_code':"games_video"}))
# len(gaming_companies) == 1083

★ Filtro el db con una query a mongodb. Le pido que me devuelva todas las compañías que no hayan quebrado y aquellas que tengan más de una oficina, que me seleccione todas.

In [5]:
company2 = list(coll.find({"$and": [{"deadpooled_year":None},{"$where":'this.offices.length >= 1'}]}))

In [6]:
len(company2)

12957

★ Creo un diccionario de una lista a la que le voy metiendo todos los valores con los que me quiero quedar.

In [7]:
merged_offices = []
for c in company2:
    offices = c.get('offices',[])
    for office in offices:
        merged_office = dict()
        raised_amount=0
        merged_office["ref"] = str(c.get('_id',0))
        merged_office["name"]=c.get('name',0)
        merged_office["category"]=c.get('category_code',0)
        merged_office["description"]=c.get('description',0)
        merged_office["latitude"]=office.get('latitude',0)
        merged_office["longitude"]=office.get('longitude',0)
        merged_office["founded_year"]=c.get('founded_year',0)
        merged_office["total_money_raised"]=c.get('total_money_raised',0)
        for rounds in range(len(c.get('funding_rounds'))):
            r=c.get('funding_rounds')[rounds].get('raised_amount',0)
            if r !=None:
                raised_amount+=r
        merged_office["raised_amount"]=raised_amount
        if len(c.get('funding_rounds'))>0:
            merged_office["currency"]=c.get('funding_rounds')[0].get('raised_currency_code',0)
        else:
            merged_office["currency"]=0
        merged_offices.append(merged_office)
        merged_office['address'] = office.get('address1', 0)
        merged_office['zip_code'] = office.get('zip_code', 0)
        merged_office['city'] = office.get('city', 0)

★ Creo un DataFrame (df) con toda esa información

In [8]:
data = merged_offices
df = pd.DataFrame(data)
df.shape

(15848, 13)

★ Defino la funcion getLocation() que me crea un diccionario en formato GeoIndex 
y mete las longitudes y latitudes de todas las oficinas de mi df

In [9]:
def getLocation(long, lat):
    loc = {
        'type':'Point',
        'coordinates':[float(long), float(lat)]
    }
    return loc

In [10]:
df_long_lat = df[['longitude', 'latitude']] 
location = []
for long,lat in df_long_lat.values:
    location.append(getLocation(long, lat))

★ Creo la columna 'GeoIndex' en mi df original con el formato {type,coordinates}

In [11]:
df['GeoIndex'] = location
# df.shape (15848 rows × 14 columns)

In [24]:
df

,ref,name,category,description,latitude,longitude,founded_year,total_money_raised,raised_amount,currency,address,zip_code,city,GeoIndex
0,52cdef7c4bab8bd675297d90,Postini,web,None,37.506885,-122.247573,1999.0,$0,0.0,USD,"959 Skyway Road, Suite 200",94070,San Carlos,"{'type': 'Point', 'coordinates': [-122.247573,..."
1,52cdef7c4bab8bd675297d92,Flektor,games_video,None,34.025958,-118.379768,NaN,$0,0.0,USD,"8536 National Blvd, Suite A",90232,Culver City,"{'type': 'Point', 'coordinates': [-118.379768,..."
2,52cdef7c4bab8bd675297d8d,Digg,news,user driven social content website,37.764726,-122.394523,2004.0,$45M,45000000.0,USD,135 Mississippi St,94107,San Francisco,"{'type': 'Point', 'coordinates': [-122.394523,..."
3,52cdef7c4bab8bd675297d91,Geni,web,Geneology social network site,34.090368,-118.393064,2006.0,$16.5M,16500000.0,USD,9229 W. Sunset Blvd.,90069,West Hollywood,"{'type': 'Point', 'coordinates': [-118.393064,..."
4,52cdef7c4bab8bd675297d95,StumbleUpon,web,Content discovery service,37.775196,-122.419204,2002.0,$18.5M,18500000.0,USD,,,San Francisco,"{'type': 'Point', 'coordinates': [-122.419204,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15843,52cdef7f4bab8bd67529c6f5,Oriact,software,Solutions for software development,47.088219,8.437163,2009.0,$0,0.0,USD,LÃ¤rchenweg 7,6403,KÃ¼ssnacht,"{'type': 'Point', 'coordinates': [8.4371634, 4..."
15844,52cdef7f4bab8bd67529c6f6,Adhunk,advertising,Online Admanager,NaN,NaN,2009.0,$0,0.0,USD,Near Tube Well(south),334003,Bikaner,"{'type': 'Point', 'coordinates': [nan, nan]}"
15845,52cdef7f4bab8bd67529c6f7,AfterLogic,software,.NET Email Components and Webmail,40.793024,-74.323554,2002.0,$0,0.0,USD,P.O. Box 2096,07039,Livingston,"{'type': 'Point', 'coordinates': [-74.3235539,..."
15846,52cdef7f4bab8bd67529c6f9,EnteGreat Solutions,software,,33.518885,-86.816068,NaN,$0,0.0,USD,,,Birmingham,"{'type': 'Point', 'coordinates': [-86.816068, ..."


★ Asigno a la columna 'currency' los simbolos de la moneda que salgan en la columna 'raised_amount'. De esta manera, evito tener "0" en 'currency'.

In [13]:
currencies = {"$":"USD", "C$":"CAD", "£":"GBP", "€":"EUR", "¥":"JPY", "kr":"SEK"}
for key,value in currencies.items():
    df.loc[(df['total_money_raised'].str.startswith(key)),'currency']=value

★ Ahora que ya tengo toda la columna 'currency' con sus respectivos valores, creo un dataframe (df_currency) que me filtre por aquellas empresas en las que el 'raised_amount' sea > 1M pero teniendo en cuenta el tipo de moneda que sea, i.e., euros, dolares, dolares canadienses, coronas suecas, etc.

In [14]:
df_currency=df[(((df['raised_amount']>=1000000) & (df['currency']=='USD'))|((df['raised_amount']>=1320000) & (df['currency']=='CAD'))|((df['raised_amount']>=900000) & (df['currency']=='EUR'))|((df['raised_amount']>=770000) & (df['currency']=='GBP'))|((df['raised_amount']>=108560000) & (df['currency']=='JPY'))|((df['raised_amount']>=9640000) & (df['currency']=='SEK')))]
df_currency

,ref,name,category,description,latitude,longitude,founded_year,total_money_raised,raised_amount,currency,address,zip_code,city,GeoIndex
2,52cdef7c4bab8bd675297d8d,Digg,news,user driven social content website,37.764726,-122.394523,2004.0,$45M,45000000.0,USD,135 Mississippi St,94107,San Francisco,"{'type': 'Point', 'coordinates': [-122.394523,..."
3,52cdef7c4bab8bd675297d91,Geni,web,Geneology social network site,34.090368,-118.393064,2006.0,$16.5M,16500000.0,USD,9229 W. Sunset Blvd.,90069,West Hollywood,"{'type': 'Point', 'coordinates': [-118.393064,..."
4,52cdef7c4bab8bd675297d95,StumbleUpon,web,Content discovery service,37.775196,-122.419204,2002.0,$18.5M,18500000.0,USD,,,San Francisco,"{'type': 'Point', 'coordinates': [-122.419204,..."
5,52cdef7c4bab8bd675297d95,StumbleUpon,web,Content discovery service,NaN,NaN,2002.0,$18.5M,18500000.0,USD,,,New York City,"{'type': 'Point', 'coordinates': [nan, nan]}"
7,52cdef7c4bab8bd675297d96,Gizmoz,web,Photo animation,37.484130,-122.169472,2003.0,$18.1M,18100000.0,USD,None,None,Menlo Park,"{'type': 'Point', 'coordinates': [-122.169472,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15828,52cdef7f4bab8bd67529c6de,Cantimer,None,,37.484616,-122.181753,2006.0,$2M,2000000.0,USD,3700 Haven Court,94025,Menlo Park,"{'type': 'Point', 'coordinates': [-122.181753,..."
15829,52cdef7f4bab8bd67529c6e0,Celestial Semiconductor,semiconductor,,37.386417,-121.929170,2004.0,$2M,2000000.0,USD,"2635 North 1st Street, Suite 150",95134,San Jose,"{'type': 'Point', 'coordinates': [-121.92917, ..."
15830,52cdef7f4bab8bd67529c6e3,InSound Medical,biotech,Hearing Devices,37.512038,-121.998513,NaN,$11M,11000000.0,USD,39660 Eureka Drive,94560,Newark,"{'type': 'Point', 'coordinates': [-121.998513,..."
15831,52cdef7f4bab8bd67529c6e4,Getyoo,mobile,NFC Events Solutions,50.839970,4.346472,2009.0,€1.13M,1130000.0,EUR,"Rue des Tanneurs, 60a",1000,Brussels,"{'type': 'Point', 'coordinates': [4.3464721, 5..."


★ Aquí simplemente estoy comprobando que me ha sustituido los "0" de la columna 'currency' por el tipo de moneda que le corresponda.

In [15]:
set(df['currency'])

{'CAD', 'EUR', 'GBP', 'JPY', 'SEK', 'USD'}

★ Divido mis preferencias en tres dataframes distintos:

➊ Dataframe con todas las compañias que hayan levantado mas de 1M.

In [25]:
dataframe_1M = df_currency
dataframe_1M.shape

(4824, 14)

➋ Dataframe con todas las compañias que han levantado mas de 1M y que además han sido creadas en los ultimos 10 años, i.e., las compañias que queremos tener cerca.

In [26]:
startups = dataframe_1M[dataframe_1M['founded_year']>=2009]
startups.shape

(159, 14)

➌ Dataframe con todas las compañias que nacieron antes de 2009 (sin tener en cuenta los $ que hayan levantado), i.e., las que queremos tener lejos.

In [27]:
old = df[df['founded_year']<=2009]
old.shape

(13239, 14)

★ Convierto los 3 dataframes a json para importarlos con mongod desde la terminal 
como collections a MongoCompass

Import Collection: mongoimport --db <database> --collection <collection_name> --jsonArray <file_name.json>

Update Collection: mongoimport --db <database> --drop --collection <collection_name> --jsonArray <file_name.json>

In [28]:
dataframe_1M.to_json('./1MillionCompanies.json', orient="records")

startups.to_json('./startups1M.json', orient="records")

old.to_json('./oldCompanies.json', orient="records")